<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Floyd-Warshall-Algorithm" data-toc-modified-id="Floyd-Warshall-Algorithm-1">Floyd-Warshall Algorithm</a></span><ul class="toc-item"><li><span><a href="#Multiprocessing-or-thread" data-toc-modified-id="Multiprocessing-or-thread-1.1">Multiprocessing or thread</a></span></li></ul></li><li><span><a href="#Johnson's-Algotirth-by-reweighting-the-edege-costs" data-toc-modified-id="Johnson's-Algotirth-by-reweighting-the-edege-costs-2">Johnson's Algotirth by reweighting the edege costs</a></span><ul class="toc-item"><li><span><a href="#Bellman-Ford-Algorithm" data-toc-modified-id="Bellman-Ford-Algorithm-2.1">Bellman-Ford Algorithm</a></span></li><li><span><a href="#Dijkstra-Algolrith" data-toc-modified-id="Dijkstra-Algolrith-2.2">Dijkstra Algolrith</a></span></li><li><span><a href="#Johnson's-Algorithm" data-toc-modified-id="Johnson's-Algorithm-2.3">Johnson's Algorithm</a></span></li><li><span><a href="#Run-Johnson's-algorithm-on-all-the-files" data-toc-modified-id="Run-Johnson's-algorithm-on-all-the-files-2.4">Run Johnson's algorithm on all the files</a></span></li></ul></li><li><span><a href="#Speed-comparison-of-list-and-numpy-array" data-toc-modified-id="Speed-comparison-of-list-and-numpy-array-3">Speed comparison of list and numpy array</a></span></li></ul></div>

In this assignment you will implement one or more algorithms for the all-pairs shortest-path problem. Here are data files describing three graphs:

g1.txt
g2.txt
g3.txt
The first line indicates the number of vertices and edges, respectively. Each subsequent line describes an edge (the first two numbers are its tail and head, respectively) and its length (the third number). NOTE: some of the edge lengths are negative. NOTE: These graphs may or may not have negative-cost cycles.

Your task is to compute the "shortest shortest path". Precisely, you must first identify which, if any, of the three graphs have no negative cycles. For each such graph, you should compute all-pairs shortest paths and remember the smallest one (i.e., compute \min_{u,v \in V} d(u,v)min 
u,v∈V
​	
 d(u,v), where d(u,v)d(u,v) denotes the shortest-path distance from uu to vv).

If each of the three graphs has a negative-cost cycle, then enter "NULL" in the box below. If exactly one graph has no negative-cost cycles, then enter the length of its shortest shortest path in the box below. If two or more of the graphs have no negative-cost cycles, then enter the smallest of the lengths of their shortest shortest paths in the box below.

OPTIONAL: You can use whatever algorithm you like to solve this question. If you have extra time, try comparing the performance of different all-pairs shortest-path algorithms!

OPTIONAL: Here is a bigger data set to play with.

large.txt
For fun, try computing the shortest shortest path of the graph in the file above.

In [1]:
from collections import defaultdict

# load data and convert to int
def read_data(filename):
    f = open(filename, 'r')
    # f.readline()
    ls = f.readline().split()
    data = []
    while ls:
        data.append([int(i) for i in ls])
        ls = f.readline().split()
    f.close()
    return data


filenames = ['g1.txt', 'g2.txt', 'g3.txt']
filename = 'g1.txt'
data = read_data(filename)
n = data[0][0]
data[:5]

[[1000, 47978], [1, 14, 6], [1, 25, 47], [1, 70, 22], [1, 82, 31]]

## Floyd-Warshall Algorithm
all pairs shortest path, valid with negative cost cycles, O(n^3) running time <br>
number vertices from 1 to n, Save the shortest path in A[i, j, k] <br>
for k from 1 to n: <br>
&emsp; for i from 1 to n: <br>
&emsp;&emsp; for j from 1 to n: <br>
&emsp;&emsp;&emsp; A[i,j,k] = min(A[i,j,k-1], A[i,k,k-1]+A[k,j,k-1] <br>
Base case:<br>
&emsp; A[i,j,0] = 0 if i = j, Cij if i,j in E, infinity if i,j not in E <br>

In [2]:
import numpy as np
import time

def floyd_warshall(filename):
    data = read_data(filename)
    n = data[0][0]

    # save all the edges in a dictionary
    edges = {(e[0],e[1]):e[2] for e in data[1:]}
    
    print('initialize cost matrix for ', filename)
    
    # initialize the cost matrix
    
    cost = np.zeros((n, n, n+1))
    for i in range(n):
        for j in range(n):
            if i != j:            
                cost[i, j, 0] = edges.get((i+1, j+1), float('inf'))
    
    print('run floyd algorithm ', filename)
    
    # run Floyd-Warshall Algorithm
    t0 = time.time()
    for k in range(1, n):
        if k % 100 == 0:
            print(filename, k)
        for i in range(n):
            for j in range(n):
                cost[i, j, k] = min(cost[i, j, k-1], cost[i, k-1, k-1] + cost[k-1, j, k-1])
            
    print(time.time() - t0)
    
    return cost
                
    # check if diagonal is negative
    
    for i in range(n):
        if cost[i, i, n] < 0:
            print(cost[i, i, n])
            return float('-inf')
                
    return cost.min()

## save results in a list in stead of np array, iteration is a little bit faster
def floyd_warshall_list(filename):
    data = read_data(filename)
    n = data[0][0]

    # save all the edges in a dictionary
    edges = {(e[0],e[1]):e[2] for e in data[1:]}
    
    print('initialize cost matrix for ', filename)
    
    # initialize the cost matrix
    
    cost = [0] * n
    for i in range(n):
        temp = [0] * n
        for j in range(n):
            temp1 = [0] * (n + 1)
            if i != j:
                temp1[0] = edges.get((i+1, j+1), float('inf'))
            temp[j] = temp1
        cost[i] = temp
    
    print('run floyd algorithm ', filename)
    
    # run Floyd-Warshall Algorithm
    t0 = time.time()
    for k in range(1, n):
        if k % 100 == 0:
            print(filename, k)
        for i in range(n):
            for j in range(n):
                cost[i][j][k] = min(cost[i][j][k-1], cost[i][k-1][k-1] + cost[k-1][j][k-1])
#                 print(i, j, k, cost[i][j][k])
            
    print(time.time() - t0)
    
    return cost
                
    # check if diagonal is negative
    
    for i in range(n):
        if cost[i][i][n] < 0:
            print(cost[i][i][n])
            return float('-inf')
                



In [3]:
costs =[]
filenames = ['g3.txt']
for filename in filenames:
    costs.append(floyd_warshall_list(filename))

# costs

initialize cost matrix for  g3.txt
run floyd algorithm  g3.txt
g3.txt 100
g3.txt 200
g3.txt 300
g3.txt 400
g3.txt 500
g3.txt 600
g3.txt 700
g3.txt 800
g3.txt 900
827.4643359184265


In [4]:
# find the minimum among all the shortest cost

for cost in costs:
#     print(min([min([(min(j) for j in i)]) for i in cost]))
    mn = float('inf')
    for i in cost:
        for j in i:
            mn = min(mn, min(j))
    print(mn)

-19


### Multiprocessing or thread
 - multithread doesn't help to spead up the calculation since it's cpu bounded. <br>
 - multiprocessing use vcore in parallel and doesn't share memory

In [5]:
import sys, threading
from multiprocessing import Process
from threading import Thread

# sys.setrecursionlimit(800000)
# threading.stack_size(67108864)

min_costs = []
threads = []


def main(filename):
    global min_costs
#     min_cost = floyd_warshall(filename)
    min_cost = 0
    min_costs.append([filename, min_cost])
    print(min_cost)

    
filenames = ['g1.txt', 'g2.txt', 'g3.txt']
for filename in filenames:
    thread = Process(target=main, args=(filename,))
    thread.daemon = True
    threads.append(thread)
    thread.start()
    
[thread.join() for thread in threads]

min_costs

for thread in threads:
    print(thread.is_alive())

0
0
0
False
False
False


## Johnson's Algotirth by reweighting the edege costs

1. Form G' by add a new vertex s to all the vertices v with edge cost of 0 
2. Run Bellman-Ford on G' with source vertex s. If detecting a negative cycle, it must be in G; report and halt.
3. For each v in G let p_v the shortest s-v path in G; for each e=(u,v) in G define c_e' = c_e + p_u - p_v
4. For each u in G: run Dijkstra's algorithm in G with edges {c_e'}, compute the shortest path d(u,v)' for each v in G
5. For each pair (u, v) in G return the shortest path d(u,v) = d(u,v)' + p_v - p_u 

### Bellman-Ford Algorithm
 - works for graphs with negative edges and negative cost cycles
 - running time O(m*n)
 - single source shortest path
1. Let A 2D array with index i and v
2. Base case: A[0,s] = 0, A[0,v] = inf for all v != s
3. for i from 1 to n-1: <br>
 &emsp; for each v in G: <br>
 &emsp; &emsp; A[i, v] = min(A[i-1,v], min(A[i-1,w] + c_wv) for w in edges of v)
 
 

In [6]:
## convert edges in the format [[w, v, cost], ....] to { v: [[w1, e1], [w2, e2], ...], v2: ...}

def to_inward_edges(data):
    edges = defaultdict(list)
    for d in data:
        edges[d[1]-1].append([d[0]-1, d[2]])  # change vertex number to 0 indexed from 1 indexed
        
    return edges

def bellman_ford(inward_edges, n, source):
    curr = [float('inf')] * n
    curr[source] = 0  
    for i in range(n):
        prev = curr[:]
        for v in range(n):
            curr[v] = min([prev[w] + cost for w, cost in inward_edges[v]] + [prev[v]])
                
    return [prev, curr]
           

### Dijkstra Algolrith
 - only works on graphs with non-negative edges
 - running time O(mlog(n)

In [7]:
# edges in a dictionary in a format v1: [[w1, ce1], [w2, ce2], [w3, ce3], ... ]
# source: starting vertex number
import heapq

def Dijkstra(edges, source):    
    X = set()  # list of vertices that have been visited
    hq = [(0, source)]  # start from source vertex with distance of 0, (dist, vertex)
    n = 0  # number of processes in the while loop
    min_cost = dict()  # vertex as the key and its shortest distance as the value, vertex: dist

    while hq:
        n += 1
        dist, v = heapq.heappop(hq)
        if v not in X:
            X.add(v) 

            # check all the neighbors of the vertex v
            for w, length in edges[v]:
                if w not in X:
                    curr = dist + length
                    if (w not in min_cost) or curr < min_cost[w]:               
                        min_cost[w] = curr
                        heapq.heappush(hq, (curr, w))

#     print(n)
    return min_cost

### Johnson's Algorithm

In [8]:
def johnson_algorithm(filename):
    
    ### Run steps 1 and 2 to calculate cost from a virtual vertex and detect negative-cost cycle
    data = read_data(filename)
    n = data[0][0]

    # add a new vertex (n+1) to G with edge cost of 0 to all the vertices from 1 to n
    for i in range(1, n+1):
        data.append([n+1, i, 0])

    inward_edges = to_inward_edges(data[1:])
    prev, curr = bellman_ford(inward_edges, n + 1, n)
    cycle = 'no negative cost cycle' if prev == curr else 'negative cost cycle found'
    
    print(filename, min(prev), min(curr), cycle)

    ### Run the rest of codes only if there is NO negative cost cycles
    
    if prev == curr:
        min_costs = [0] * n  # 2D array to store minimum costs of the path for all possible (u,v) pairs
        
        ## Step 3: reweight all the edges to make them non-negative
        # save all the edges in a dictionary in a format v1: [[w1, ce1], [w2, ce2], [w3, ce3], ... ]
        # reweight edge cost with Bellman-Ford results
        
        edges = defaultdict(list)
        for d in data[1:]:
            new_edge = d[2] + curr[d[0]-1] - curr[d[1]-1]
            if new_edge < 0:
                print(new_edge)
            edges[d[0]-1].append([d[1]-1, new_edge])  # change to 0 indexed from 1-indexed
        
        ## Step 4 and 5: run Dijkstra algorithm on each v in G' and caculate the original minimum costs 
        
        for i in range(n):
            min_cost = Dijkstra(edges, i)
            new_min = [ 0 ] * n  # orignal min_cost of the path from i to all the other vertices
            for v in min_cost:
                new_min[v] = min_cost[v] + curr[v] - curr[i]
            min_costs[i] = new_min
            
        return min_costs
    
    return []

### Run Johnson's algorithm on all the files

In [9]:
# run Johnson's algorithm on three small files
def main(filename):
    min_costs = johnson_algorithm(filename)
    min_cost = min([min(c) for c in min_costs]) if min_costs else float('inf')
    print('shortest shortest cost: ', min_cost)
    
    # find start and end vertex of the shortest shortest cost
    if min_costs:
        for i, cost in enumerate(min_costs):
            for j, c in enumerate(cost):
                if c == min_cost:
                    print('start and end vertices of the shortest path:', i, j)
                    
filenames = ['g1.txt', 'g2.txt', 'g3.txt']
for filename in filenames:
    main(filename)

g1.txt -1749 -1750 negative cost cycle found
shortest shortest cost:  inf
g2.txt -1547 -1548 negative cost cycle found
shortest shortest cost:  inf
g3.txt -19 -19 no negative cost cycle
shortest shortest cost:  -19
start and end vertices of the shortest path: 398 903


In [10]:
### Run Johnson's algorithm on the large file; May take 7 hours to finish

# main('large.txt')

## Speed comparison of list and numpy array

In [11]:
## speed comparison of different implementation of lists

def list_1(n): 
    cost = [0] * n 
    for i in range(n):
        temp0 = [0] * n
        for j in range(n):
            temp0[j] =  [10*i + j if i != j else 0] + [0] * n  # concatenate list
        cost[i] = temp0

def list_2(n):
    cost = [0] * n 
    for i in range(n):
        temp0 = [0] * n
        for j in range(n):
            temp = [0] * (n + 1)
            temp[0] = 10*i + j if i != j else 0
            temp0[j] = temp
        cost[i] = temp0
        
def list_3(n): 
    cost = []
    for i in range(n):
        temp0 = []
        for j in range(n):
            temp = [0] * (n + 1)
            temp[0] = 10*i + j if i != j else 0
            temp0.append(temp)  # use append
        cost.append(temp0)    # use append

n = 300
%timeit list_1(n)

%timeit list_2(n)

%timeit list_3(n)

254 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
135 ms ± 5.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
135 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
b = np.zeros((n,n,n))
a = b.tolist()

In [13]:
def t_array(n):
    for k in range(n):
        for i in range(n):
            for j in range(n):
                b[i,j,k] = min(b[i,j,k-1], b[i, k-1, k-1] + b[k-1, j, k-1])

def t_list(n):
    for k in range(n):
        for i in range(n):
            for j in range(n):
                a[i][j][k] = min(a[i][j][k-1], a[i][k-1][k-1] + a[k-1][j][k-1])

n = 100
%timeit t_array(n)
%timeit t_list(n)

1.19 s ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
734 ms ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
n

100